In [1]:
import merlin.models.tf as ml
from merlin.datasets.synthetic import generate_data

social_data = generate_data("social", num_rows=1000)

social_data.to_ddf()

2022-02-17 11:07:50.170973: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-17 11:07:51.695487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46352 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-02-17 11:07:51.696901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46363 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:21:00.0, compute capability: 8.6


,user_id,user_categories,user_intentions,user_profile,user_group,user_gender,user_age,user_consumption_1,user_is_occupied,user_geography,item_id,item_category,item_user_id,click,like,comment,share,hide,position
0,20,392,2176,7,1,1,1,1,1,1,12,189,6459,1,1,0,1,0,1
1,8,145,802,3,1,1,1,1,1,1,35,582,19963,1,1,0,0,0,1
2,147,3012,16721,49,7,1,4,2,1,2,7,103,3523,0,1,1,1,1,2
3,42,846,4696,14,2,1,1,1,1,1,217,3692,126819,0,1,1,0,1,1
4,23,454,2520,8,1,1,1,1,1,1,16,257,8807,0,1,0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,65,1321,7330,22,3,1,2,1,1,1,68,1146,39338,0,1,0,0,0,3
996,14,269,1489,5,1,1,1,1,1,1,5,69,2349,0,0,0,0,1,3
997,12,227,1260,4,1,1,1,1,1,1,71,1197,41099,0,0,1,1,1,2
998,73,1486,8246,24,4,1,2,1,1,1,53,889,30531,0,0,1,1,1,3


## Retrieval

In [2]:
retrieval_model = ml.TwoTowerModel(social_data.schema, query_tower=ml.MLPBlock([512, 256]))

retrieval_model.compile(optimizer="adam", run_eagerly=True)
retrieval_model.fit(social_data, batch_size=50, epochs=2)

print(f"Number of parameters: {retrieval_model.count_params()}")

Epoch 1/2


2022-02-17 11:07:57.391187: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-02-17 11:07:58.359543: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


20/20 [==============================] - 8s 240ms/step - ndcg_at_1_10: 0.1113 - ndcg_at_1_20: 0.1572 - recall_at_1_10: 0.2210 - recall_at_1_20: 0.4029 - avg_precision_at_1_10: 0.0788 - avg_precision_at_1_20: 0.0914 - loss: 3.8922 - regularization_loss: 0.0000e+00 - total_loss: 3.8922
Epoch 2/2
20/20 [==============================] - 5s 241ms/step - ndcg_at_1_10: 0.2854 - ndcg_at_1_20: 0.3357 - recall_at_1_10: 0.5010 - recall_at_1_20: 0.7010 - avg_precision_at_1_10: 0.2204 - avg_precision_at_1_20: 0.2340 - loss: 3.8207 - regularization_loss: 0.0000e+00 - total_loss: 3.8207
Number of parameters: 238537970


## Ranking

In [4]:
ranking_model = ml.DLRMModel(
    social_data.schema,
    embedding_dim=64,
    bottom_block=ml.MLPBlock([64]),
    top_block=ml.MLPBlock([32]),
)

ranking_model.compile(optimizer="adam", run_eagerly=True)
ranking_model.fit(social_data, batch_size=50, epochs=2)

print(f"Number of parameters: {ranking_model.count_params()}")

Epoch 1/2
20/20 [==============================] - 2s 112ms/step - click/binary_classification_task/precision: 0.4720 - click/binary_classification_task/recall: 0.3052 - click/binary_classification_task/binary_accuracy: 0.4840 - click/binary_classification_task/auc: 0.4846 - like/binary_classification_task/precision: 0.5052 - like/binary_classification_task/recall: 0.5746 - like/binary_classification_task/binary_accuracy: 0.5030 - like/binary_classification_task/auc: 0.5120 - comment/binary_classification_task/precision: 0.4211 - comment/binary_classification_task/recall: 0.0177 - comment/binary_classification_task/binary_accuracy: 0.5440 - comment/binary_classification_task/auc: 0.4972 - share/binary_classification_task/precision: 0.4728 - share/binary_classification_task/recall: 0.4038 - share/binary_classification_task/binary_accuracy: 0.5050 - share/binary_classification_task/auc: 0.4791 - hide/binary_classification_task/precision: 0.5015 - hide/binary_classification_task/recall: 0